# Tuto to find The DIA sources and DIA Objects in LSSTComCam

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- member : DESC, rubin-inkind
- creation date : 2025-05-02
- last update : 2025-05-02

## Note on work done on LSSTComCam Commissioning : https://sitcomtn-149.lsst.io/
## Note Data Product definition Document : https://lse-163.lsst.io/

In [ ]:
import sys
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
import numpy as np
import pandas as pd
from astropy.time import Time
# %matplotlib widget

In [ ]:
afwDisplay.setDefaultBackend("firefly")

In [ ]:
import lsst.geom as geom
from lsst.geom import SpherePoint, Point2D, Point2I, degrees
from lsst.skymap import PatchInfo, Index2D

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "x-large"
plt.rcParams["axes.titlesize"] = "x-large"
plt.rcParams["xtick.labelsize"] = "x-large"
plt.rcParams["ytick.labelsize"] = "x-large"

In [ ]:
import gc

In [ ]:
import traceback

In [ ]:
# Define butler
from lsst.daf.butler import Butler

In [ ]:
!eups list lsst_distrib

In [ ]:
def nJy_to_ab_mag(f_njy):
    """Convert flux in nanojanskys to AB magnitude."""
    if f_njy > 0:
        return -2.5 * np.log10(f_njy) + 31.4
    else:
        return 0.0


def ab_mag_to_nJy(mag_ab):
    """Convert AB magnitude to flux in nanojanskys."""
    return 10 ** ((31.4 - mag_ab) / 2.5)

## RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

## Configuration

In [ ]:
FLAG_DUMP_COLLECTIONS = False
FLAG_DUMP_DATASETS = True
FLAG_DUMP_OBJECTSTABLECOLUMNS = True
FLAG_CUT_OBJECTSMAG = True

In [ ]:
MAGCUT = 24.0
NDIACUT = 100

In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]
band_to_col = dict(u="b", g="g", r="r", i="orange", z="grey", y="k")
all_bands_colors = ["blue", "green", "red", "orange", "yellow", "purple"]

### Choose instrument

In [ ]:
# instrument = "LSSTCam"

# We focus here on sky fields oberved by LSSTComCam, so we select this camera
instrument = "LSSTComCam"

### Choose options

### For LSSTCam : RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

### For LSSTComCam check here : 
- Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

In [ ]:
if instrument == "LSSTCam":
    repo = "/repo/embargo"
    instrument = "LSSTCam"
    collection_validation = instrument + "/runs/nightlyValidation"
    # collection_quicklook   = instrument + '/runs/quickLookTesting'
    collection_validation = os.path.join(collection_validation, "20250416/d_2025_04_15/DM-50157")
    date_start = 20250415
    date_selection = 20250416
    where_clause = "instrument = '" + f"{instrument}" + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"
    skymapName = "lsst_cells_v1"
    BANDSEL = "i"

elif instrument == "LSSTComCam":
    repo = "/repo/main"
    collection_validation = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"  # work 2025-05-01
    # collection_validation = "LSSTComCam/runs/DRP/DP1/w_2025_17/DM-50530" #updated 2025-05-02
    date_start = 20241024
    date_selection = 20241211
    skymapName = "lsst_cells_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063
    BANDSEL = "i"

In [ ]:
collectionStr = collection_validation.replace("/", "_")

## Access to Butler registry

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection_validation)
registry = butler.registry

## Create a skymap object and Camera

In [ ]:
skymap = butler.get("skyMap", skymap=skymapName, collections=collection_validation)

In [ ]:
camera = butler.get("camera", collections=collection_validation, instrument=instrument)

## Query for collections in Butler

- remove user collections
- remove calibration products

In [ ]:
# mostly setup for LSSTCam
if FLAG_DUMP_COLLECTIONS:
    for _ in sorted(registry.queryCollections(expression=instrument + "/*")):
        if "/calib/" not in _ and "u/" not in _:
            print(_)

## Query for the dataset types in the Butler

- Refer to the Data Product definition Document to know about the definition of datasets
- https://www.lsst.org/about/dm/data-products
- https://lse-163.lsst.io/
- https://docushare.lsst.org/docushare/dsweb/Get/LSE-163

In [ ]:
if FLAG_DUMP_DATASETS:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection_validation).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and ("Plot" not in datasetType.name)
                and ("Metric" not in datasetType.name)
                and ("metric" not in datasetType.name)
            ):
                if "object" in datasetType.name or "Obj" in datasetType.name:
                    print(datasetType)
                if "source" in datasetType.name or "Source" in datasetType.name:
                    print(datasetType)

### Thus Selection of Datasets 

    DatasetType('diaSourceTable_tract', {skymap, tract}, DataFrame)
    DatasetType('diaObjectTable_tract', {skymap, tract}, DataFrame)
    DatasetType('forcedSourceOnDiaObjectTable', {skymap, tract, patch}, DataFrame)
    DatasetType('forcedSourceOnDiaObjectTable', {skymap, tract, patch}, DataFrame)
    DatasetType('mergedForcedSourceOnDiaObject', {band, instrument, skymap, day_obs, detector, physical_filter, tract, visit}, DataFrame)
    DatasetType('forcedSourceTable', {skymap, tract, patch}, DataFrame)

## Selection of Region of interest

In [ ]:
lsstcomcam_targets = {}
lsstcomcam_targets["47 Tuc"] = {"field_name": "47 Tuc Globular Cluster", "ra": 6.02, "dec": -72.08}
lsstcomcam_targets["Rubin SV 38 7"] = {"field_name": "Low Ecliptic Latitude Field", "ra": 37.86, "dec": 6.98}
lsstcomcam_targets["Fornax dSph"] = {
    "field_name": "Fornax Dwarf Spheroidal Galaxy",
    "ra": 40.0,
    "dec": -34.45,
}
lsstcomcam_targets["ECDFS"] = {"field_name": "Extended Chandra Deep Field South", "ra": 53.13, "dec": -28.10}
lsstcomcam_targets["EDFS"] = {"field_name": "Euclid Deep Field South", "ra": 59.10, "dec": -48.73}
lsstcomcam_targets["Rubin SV 95 -25"] = {
    "field_name": "Low Galactic Latitude Field",
    "ra": 95.00,
    "dec": -25.0,
}
lsstcomcam_targets["Seagull"] = {"field_name": "Seagull Nebula", "ra": 106.23, "dec": -10.51}

In [ ]:
# the_target = lsstcomcam_targets["Seagull"]
# the_target = lsstcomcam_targets["47 Tuc"] # bad
# the_target = lsstcomcam_targets["Fornax dSph"]
# the_target = lsstcomcam_targets["ECDFS"]


# key = "Seagull"
# key = "Fornax dSph"
key = "ECDFS"
# key = "EDFS"
# key = "47 Tuc"
# key = "Rubin SV 38 7"
# key = "Rubin SV 95 -25"


the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_title = (
    the_target["field_name"] + f" band  {BANDSEL} " + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)
target_point = SpherePoint(target_ra, target_dec, degrees)

## Get list of tracts from the objectTable_tract

In [ ]:
datasettype = "diaObjectTable_tract"

In [ ]:
# Find the dimension
print(butler.registry.getDatasetType(datasettype).dimensions)

In [ ]:
therefs = butler.registry.queryDatasets(datasettype, collections=collection_validation)
tractsId_list = np.unique([ref.dataId["tract"] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

## Find the Tract and Patch of the region of interest

- tract in tractNbSel
- patch in patchNbSel

In [ ]:
tract_info = skymap.findTract(target_point)
patch_info = tract_info.findPatch(target_point)
bbox = patch_info.getOuterBBox()

print("Patch bounding box:", bbox)

print("Tract ID :", tract_info.getId())
tractNbSel = tract_info.getId()

print("Patch Index :", patch_info.getIndex(), " , ", patch_info.getSequentialIndex())  # (x, y)
print("Bounding Box", bbox)

patchNbSel = patch_info.getSequentialIndex()

In [ ]:
dataId = {"band": BANDSEL, "tract": tractNbSel, "patch": patchNbSel, "skymap": skymapName}

In [ ]:
patch_info

## The Objects

- Objects are extracted from object detection on deepcoadds
- all bands are included 

In [ ]:
print(butler.registry.getDatasetType(datasettype).dimensions)

In [ ]:
# cannot add a filter on band
where_clause = f"skymap = '{skymapName}' AND tract = {tractNbSel} AND patch = {patchNbSel}"
print(where_clause)

In [ ]:
dataset_refs = list(
    butler.registry.queryDatasets(datasettype, collections=collection_validation, where=where_clause)
)
# Récupère un des refs valides
Nrefs = len(dataset_refs)
print(f"Number of DIAobjectTables : {Nrefs}")
ref = dataset_refs[0]
t = butler.get(ref)

Nobj = len(t)
# del t
# gc.collect()
print(f"Total Number of DIAobjects {Nobj}")

In [ ]:
t

Oui, dans ce type de objectTable, les colonnes comme g_psfFlux, g_kronFlux, g_cModelFlux, etc., sont des flux calibrés (en nJy ou en unité de calibration interne du pipeline). Pour les convertir en magnitudes AB, tu peux utiliser la formule classique :

In [ ]:
# Utilise ref.datasetType.name pour lister les colonnes disponibles
if FLAG_DUMP_OBJECTSTABLECOLUMNS:
    t_columns = list(butler.get(ref).columns)
    print(t_columns)

In [ ]:
# La constante 31.4 correspond à la conversion AB standard pour un flux exprimé en nanoJanskys (nJy).
# À vérifier selon l’unité exacte utilisée par le pipeline sur ton RSP
# (souvent c’est bien nJy, mais ça peut être autre chose si la calibration a été changée).
# mag = -2.5 * np.log10(flux) + 31.4

In [ ]:
# Move the object id in the columns to keep memory of object id
t = t.reset_index()

In [ ]:
t

## Extract info 

In [ ]:
# Check columns definitions in https://lse-163.lsst.io/
all_radecTable = []

for band in all_bands:
    id_name = "diaObjectId"
    coord_ra_name = "ra"
    coord_dec_name = "dec"
    nsources_name = "nDiaSources"
    psfflux_name = f"{band}_psfFluxMean"
    psfmag_name = f"{band}_psfMagMean"
    df = t[[id_name, coord_ra_name, coord_dec_name, nsources_name, psfflux_name]]
    # compute magnitude AB
    # df[psfmag_name] = df[psfflux_name].apply(nJy_to_ab_mag)
    # Apparemment il faut faire une copie de df sinon pandas rale

    df_sub = df.copy()
    df_sub[psfmag_name] = df_sub[psfflux_name].apply(nJy_to_ab_mag)
    # select bright objects
    if FLAG_CUT_OBJECTSMAG:
        df_sub = df_sub[df_sub[psfmag_name] < MAGCUT]

    df_sub = df_sub[df_sub[psfmag_name] > 0]

    # check of object is in selected patch
    all_flags = []
    all_x = []
    all_y = []
    for index, row in df_sub.iterrows():
        object_point = SpherePoint(row[coord_ra_name], row[coord_dec_name], degrees)
        inTractFlag = tract_info.contains(object_point)
        # Does not exist:
        # inPatchFlag = patch_info.contains(object_point)

        # the wcs from the tract
        the_wcs = tract_info.getWcs()
        # the bbox of the patch
        the_bbox = patch_info.getOuterBBox()
        # Convertir coordonnée céleste → pixel (coord tract + patch)
        pixel = the_wcs.skyToPixel(object_point)
        x_pix = Point2I(pixel).x
        y_pix = Point2I(pixel).y
        all_x.append(x_pix)
        all_y.append(y_pix)
        inPatchFlag = the_bbox.contains(Point2I(pixel))

        all_flags.append(inPatchFlag)
    df_sub["inPatch"] = all_flags
    df_sub["x"] = all_x
    df_sub["y"] = all_y

    # save table in the list
    all_radecTable.append(df_sub)

## Check position of sources of patch

In [ ]:
fig, axis = plt.subplots(2, 3, figsize=(16, 6), sharex=True, sharey=True, layout="constrained")

axs = axis.flatten()

for idx, ax in enumerate(axs):
    band = all_bands[idx]
    col = band_to_col[band]
    t = all_radecTable[idx]
    t.plot.scatter(x=coord_ra_name, y=coord_dec_name, ax=ax, color="cyan", marker=".", s=1, lw=1, alpha=0.5)
    t[t["inPatch"]].plot.scatter(
        x=coord_ra_name, y=coord_dec_name, ax=ax, color=col, marker=".", s=1, lw=1, label=band, alpha=1
    )
    ax.set_aspect("equal")

ax.invert_xaxis()
plt.suptitle(
    f"DIA objects position in tract {tractNbSel} and patch {patchNbSel}", fontsize=20, fontweight="bold"
)

plt.show()

## Number of sources per objects

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
t["nDiaSources"].hist(ax=ax, bins=100, histtype="step", color="k", range=(0, 800))
ax.set_yscale("log")
ax.set_title(f"Number of sources per DIA objects in tract {tractNbSel}")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 6), sharex=True, sharey=True, layout="constrained")

ax1, ax2 = axs.flatten()

for count, band in enumerate(all_bands):
    nsources_name = "nDiaSources"
    psfmag_name = f"{band}_psfMagMean"
    col = band_to_col[band]
    t = all_radecTable[count]
    t.plot.scatter(
        x=nsources_name, y=psfmag_name, ax=ax1, color=col, marker="o", s=30, lw=3, label=band, alpha=0.5
    )
    ax1.set_title(f"Magnitude per Nobs DIA objects in tract {tractNbSel}")
    t[t["inPatch"]].plot.scatter(
        x=nsources_name, y=psfmag_name, ax=ax2, color=col, marker="o", s=30, lw=3, label=band, alpha=0.5
    )
    ax2.set_title(f"Magnitude per Nobs DIA objects in tract {tractNbSel},patch {patchNbSel}")

plt.show()

## Select above nDIASources> NDIACUT

In [ ]:
all_radecTable_sel = []

for count, band in enumerate(all_bands):
    t = all_radecTable[count]
    t = t[t["inPatch"]]
    t = t[t[nsources_name] > NDIACUT]
    all_radecTable_sel.append(t)

In [ ]:
t

## Get Coadds and WCS

In [ ]:
all_dataIds = [
    {
        "band": band,
        "tract": tractNbSel,
        "patch": patchNbSel,
        "skymap": skymapName,
    }
    for band in all_bands
]

In [ ]:
all_deepCoadds = []
all_deepCoadds_wcs = []

all_titles = []
for idx, band in enumerate(all_bands):
    the_band = band
    the_dataId = all_dataIds[idx]
    the_title = key + f" band {the_band}"
    try:
        coadd_exp = butler.get("deepCoadd_calexp", the_dataId)
        wcs = coadd_exp.getWcs()
        psf = coadd_exp.getPsf()
        all_deepCoadds.append(coadd_exp)
        all_deepCoadds_wcs.append(wcs)
        all_titles.append(the_title)

    except Exception as inst:
        print(f"{key} :: catch Exception for band {band}")
        print(type(inst))  # the exception type
        print(inst.args)  # arguments stored in .args
        print(inst)  # __str_

In [ ]:
N = len(all_deepCoadds)
for count in range(N):
    band = all_bands[count]
    display = afwDisplay.Display(frame=count + 1)
    # cannot succeed to show white stars on dark sky
    display.setImageColormap("gray")
    display.scale("asinh", "zscale")
    display.mtv(all_deepCoadds[count].image, title=all_titles[count])

    # work on targets
    wcs = all_deepCoadds_wcs[count]
    df = all_radecTable_sel[count]
    x_name = "x"
    y_name = "y"

    with display.Buffering():
        for index, row in df.iterrows():
            xpix = row[x_name]
            ypix = row[y_name]
            display.dot("o", xpix, ypix, size=50, ctype=all_bands_colors[count])

## Objects Ids

Make list of dictionnary of objects numbers

In [ ]:
listOfObjIds = []
dictOfObjIds = {}
for count, band in enumerate(all_bands):
    t = all_radecTable_sel[count]
    obj_ids = np.sort(t[id_name].values)
    n = len(obj_ids)
    listOfObjIds.append(obj_ids)
    dictOfObjIds[band] = obj_ids
    print(f"band {band} nobj = {n} : ", obj_ids)

In [ ]:
all_diaobj_ids = np.unique(np.sort(np.concat(listOfObjIds)))
all_diaobj_ids

## Find Sources associated to the DIA object selected

In [ ]:
datasettype = "diaSourceTable_tract"

In [ ]:
dataset_refs = list(
    butler.registry.queryDatasets(datasettype, collections=collection_validation, where=where_clause)
)
# Récupère un des refs valides
Nrefs = len(dataset_refs)
print(f"Number of DIAsourceTables : {Nrefs}")
ref = dataset_refs[0]
t = butler.get(ref)

Nsrc = len(t)
# del t
# gc.collect()
print(f"Total Number of DIAsouurces {Nsrc}")

In [ ]:
t.head()

### Make lists of sources for each objects

In [ ]:
all_objsrc = []

for diaobj_id in all_diaobj_ids:
    t_sel = t[t.diaObjectId == diaobj_id]
    all_objsrc.append(t_sel)

In [ ]:
print(list(all_objsrc[0].columns))

In [ ]:
# release memory used by the source table
del t
gc.collect()

In [ ]:
all_objsrc[0][
    [
        "apFlux",
        "apFluxErr",
        "snr",
        "psfFlux",
        "psfFluxErr",
        "band",
        "midpointMjdTai",
        "diaObjectId",
        "ssObjectId",
        "parentDiaSourceId",
    ]
]

In [ ]:
def remove_outliers_iqr(x, y, yerr, factor=1.5):
    q1, q3 = np.percentile(y, [25, 75])
    iqr = q3 - q1
    low, high = q1 - factor * iqr, q3 + factor * iqr
    mask = (y >= low) & (y <= high)
    return x[mask], y[mask], yerr[mask]

In [ ]:
flux_name = "apFlux"
fluxerr_name = flux_name + "Err"

for idx_obj, diaobj_id in enumerate(all_diaobj_ids):
    ts = all_objsrc[idx_obj]
    fig, ax = plt.subplots(1, 1, figsize=(16, 3))
    title = f"{idx_obj}) {datasettype} for DiaObject = {diaobj_id} in tract {tractNbSel},patch {patchNbSel}"

    for ib, band in enumerate(all_bands):
        if diaobj_id in dictOfObjIds[band]:
            tsb = ts[ts["band"] == band]
            # print(f"object {diaobj_id} in band {band}")
            x = tsb["midpointMjdTai"].values + 0.2 * ib
            y = tsb[flux_name].values
            yerr = tsb[fluxerr_name].values
            # remove outliers
            x, y, yerr = remove_outliers_iqr(x, y, yerr)

            ax.errorbar(
                x,
                y,
                yerr=yerr,
                marker="o",
                lw=0,
                mfc=band_to_col[band],
                mec=band_to_col[band],
                ecolor=band_to_col[band],
                label=band,
            )
    ax.legend()
    ax.set_xlabel("mjd")
    ax.set_ylabel(flux_name)
    ax.set_title(title)
    # ax.set_yscale("symlog")

## Find Forced Sources on DiaObjects

In [ ]:
datasettype = "forcedSourceOnDiaObjectTable"
# datasettype = "mergedForcedSourceOnDiaObject"

In [ ]:
dataset_refs = list(
    butler.registry.queryDatasets(datasettype, collections=collection_validation, where=where_clause)
)
# Récupère un des refs valides
Nrefs = len(dataset_refs)
print(f"Number of Foorced DIAsourceTables : {Nrefs}")
ref = dataset_refs[0]
t = butler.get(ref)

Nsrc = len(t)
# del t
# gc.collect()
print(f"Total Number of Forced DIAsources {Nsrc}")

In [ ]:
print(list(t.columns))

- Il faut trouver le temps associé à une visite comme dans mon notebook : https://github.com/sylvielsstfr/mySIT-COM2025/blob/main/notebooks/2025-04-LSSTComCam/2025-04-21-SearchAGNInLSSTComCamSources.ipynb

### Make lists of forced sources for each objects

In [ ]:
all_objfsrc = []

for diaobj_id in all_diaobj_ids:
    t_sel = t[t.diaObjectId == diaobj_id]
    all_objfsrc.append(t_sel)

In [ ]:
print(list(all_objfsrc[0].columns))

In [ ]:
# release memory used by the source table
del t
gc.collect()

In [ ]:
all_objfsrc[0][
    [
        "apFlux",
        "apFluxErr",
        "snr",
        "psfFlux",
        "psfFluxErr",
        "band",
        "midpointMjdTai",
        "diaObjectId",
        "ssObjectId",
        "parentDiaSourceId",
    ]
]